Notebook written by Bjorn Larsen - bjorn.larsen@nanograv.org

This was written on short notice, please ask if something is confusing or not working (or incorrect)!

In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

In [ ]:
import numpy as np
import scipy
import os, pickle, logging, copy, json, git
logging.basicConfig(level=logging.WARNING)

In [ ]:
import astropy.units as u
import pint_pal.lite_utils as lu
import pint_pal.plot_utils as pu
from pint_pal.timingconfiguration import TimingConfiguration
from enterprise.pulsar import Pulsar
from enterprise.pulsar import BasePulsar
from enterprise.signals import parameter, signal_base, gp_priors, gp_signals, white_signals, utils
import enterprise.constants as const
from enterprise_extensions.sampler import setup_sampler, get_parameter_groups

In [ ]:
import la_forge.core as co
import la_forge.diagnostics as dg
import corner

# Intro to IPTA-style noise analysis

This notebook introduces us to the process of performing noise analysis for IPTA style datasets. We generally categorizing timing noise as correlated changes in our timing residuals which are *stochastic* (effectively random based on our information about the system). Timing noise arises primarily due to changes in the ionized plasma of the interstellar medium (ISM) along the Earth-pulsar line of sight, spin fluctuations in the pulsar, or from gravitational waves. From the data analysis standpoint, we categorize different noises sources in terms of their *spectrum* (what we get if we take the Fourier transform of the timing residuals induced by the noise process) and their *chromaticity* (how does the amplitude of the noise vary as a function of the observed radio frequency). Ultimately we have to define some type of parameterized model that is flexible enough to fit for these processes without overfitting our data. For this we will move into using the `enterprise` software for fitting the noise processes, however in the end we can also include these noise processes to update our timing model fits in `pint`.

To get a feel for the general process, let's first look at a toy dataset which contains an example of correlated noise, but not timing model effects.

## Generate simplified data

Here we will ignore the contributions of the timing model, and simply generate some timing residuals from scratch to illustrate the types of noise we will be looking at.

First we ought to define the noise parameters ought to infer. We will have 3 white noise parameters (`efac`, `equad`, `ecorr`), 2 red noise parameters (`red_noise_A`, `red_noise_gam`), and 2 dm noise parameters (`dm_noise_A`, `dm_noise_gam`) which will introduce radio-frequency dependent noise. A and gamma describe the parameters of a power law spectrum in Fourier space.

In [ ]:
# first define noise parameters
efac_true = 1.1
equad_true = 0.3 # us
ecorr_true = 0.5 # us
red_noise_A_true = 1 # us at fref
red_noise_gam_true = 4
dm_noise_A_true = 0.5 # us at fref and nu = 1400 MHz
dm_noise_gam_true = 2

In [ ]:
# params describe our toy data
Nep = 200 # number of epochs
Nnu = 4 # number of radio frequency observations per epoch
Ntoas = Nep*Nnu
Tmin = 0 # yr
Tmax = 15 # yr
Tspan = Tmax - Tmin
nu_min = 800 # MHz
nu_max = 2200 # MHz
fref = 1/Tspan # 1/yr

Next let's generate some data

In [ ]:
np.random.seed(1000)

# generate randomly distributed data
toas = np.sort(np.random.uniform(Tmin, Tmax, size=Nep)).repeat(Nnu)

# define radio freuqencies of the TOAs, uniformly spaced from nu_min to nu_max
freqs = np.array(list(np.linspace(nu_min, nu_max, Nnu))*Nep)

# generate randomly distributed errors
toaerr = np.random.uniform(0.1, 1, size=Ntoas)

# generate Gaussian errors
n = np.random.normal(size=Ntoas)

# scale by the toa errors
resids_noiseless = n*toaerr

# plot the data with errorbars
plt.figure(figsize=(8,4))
plt.errorbar(toas, resids_noiseless, yerr=toaerr, fmt='.k', alpha=0.3, ms=10, zorder=1)
# plot the data as a scatterplot with colors on top to show radio frequencies
sc = plt.scatter(toas, resids_noiseless, c=freqs, cmap='Spectral', alpha=1, s=10,
                 vmin=nu_min, vmax=nu_max, zorder=2)
cbar = plt.colorbar(mappable=sc)
cbar.ax.set_ylabel('Radio frequency (MHz)')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Residuals ($\mu$s)')
plt.show()

This may be what our pulsar timing residuals look like after a good timing model fit. The errorbars all have different sizes, but if we divide the data by our errorbars we can see the data looks Gaussian.

In [ ]:
# PDF of the normal distribution
grid = np.linspace(-4,4,500)
gauss_pdf = scipy.stats.norm().pdf(grid)

# plot the data with errorbars
fig, axes = plt.subplots(1,2,figsize=(8,4),sharey=True,width_ratios=[3,1])
axes[0].scatter(toas, resids_noiseless/toaerr, c='k', alpha=1, s=10)
axes[1].hist(resids_noiseless/toaerr, bins=30, color='k', alpha=0.5, orientation='horizontal', density=True)
axes[1].plot(gauss_pdf, grid, '-k')
axes[0].set_xlabel('TOAs (yr)')
axes[0].set_ylabel(r'Normalized Residuals')
fig.subplots_adjust(wspace=0)
plt.show()

We have not added in any of our noise yet. First we will scale the noiseless residuals by some amount, which would correspond to what we would get if we underestimated all the errorbars. This is what we assume when we include the EFAC parameter in the model.

In [ ]:
resids_with_efac = resids_noiseless*efac_true

We will also add in some errors in quadrature. We could do this in two ways, either by regenerating the data following the formula
\begin{align}
    \sigma^2 = \mathcal{F}^2\sigma_{\mathrm{ToA}}^2 + \mathcal{Q}^2,
\end{align}
where $\mathcal{F}$ is EFAC and $\mathcal{Q}$ is EQUAD, or we could just add independent measurement noise

In [ ]:
n_independent = equad_true*np.random.normal(size=Ntoas)

We can see now that the normalized residuals with this extra white noise is not a perfect match for the Gaussian

In [ ]:
resids_with_measurement_noise = resids_with_efac + n_independent

fig, axes = plt.subplots(1,2,figsize=(8,4),sharey=True,width_ratios=[3,1])
axes[0].scatter(toas, resids_with_measurement_noise/toaerr, c='k', alpha=1, s=10)
axes[1].hist(resids_with_measurement_noise/toaerr, bins=30, color='k',
             alpha=0.5, orientation='horizontal', density=True)
axes[1].plot(gauss_pdf, grid, '-k')
axes[0].set_xlabel('TOAs (yr)')
axes[0].set_ylabel(r'Normalized Residuals')
fig.subplots_adjust(wspace=0)
plt.show()

Updating our errorbars accordingly will correct the distribution

In [ ]:
toaerrs_with_measurement_noise = np.sqrt(efac_true**2*toaerr**2 + equad_true**2)

fig, axes = plt.subplots(1,2,figsize=(8,4),sharey=True,width_ratios=[3,1])
axes[0].scatter(toas, resids_with_measurement_noise/toaerrs_with_measurement_noise, c='k', alpha=1, s=10)
axes[1].hist(resids_with_measurement_noise/toaerrs_with_measurement_noise, bins=30, color='k',
             alpha=0.5, orientation='horizontal', density=True)
axes[1].plot(gauss_pdf, grid, '-k')
axes[0].set_xlabel('TOAs (yr)')
axes[0].set_ylabel(r'Normalized Residuals')
fig.subplots_adjust(wspace=0)
plt.show()

That nice, but what if we have some type of white noise that affects all of the TOAs in an observation at once? Any type of astrophysical white noise will have this effect. One example is *pulse jitter*, which is caused by random variations in the pulse's shape. Possibly, we could get a similar effects from some short-timescale ISM processes (although this will also be *chromatic*).

In [ ]:
# add jitter
jitter = ecorr_true*np.random.normal(size=Nep).repeat(Nnu)

We can distinguish visually the difference between the independent measurement noise and jitter noise.

In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(toas, n_independent, c='k', alpha=1, s=10, label='Measurement noise (EQUAD)')
plt.scatter(toas, jitter, c='C0', alpha=1, s=10, label='Jitter noise (ECORR)')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Residuals ($\mu$s)')
plt.show()

Here's the total set of residuals

In [ ]:
resids_with_white_noise = resids_with_measurement_noise + jitter

# plot the data with errorbars
plt.figure(figsize=(8,4))
plt.errorbar(toas, resids_with_white_noise, yerr=toaerr, fmt='.k', alpha=0.3, ms=10, zorder=1)
# plot the data as a scatterplot with colors on top to show radio frequencies
sc = plt.scatter(toas, resids_with_white_noise, c=freqs, cmap='Spectral', alpha=1, s=10,
                 vmin=nu_min, vmax=nu_max, zorder=2)
cbar = plt.colorbar(mappable=sc)
cbar.ax.set_ylabel('Radio frequency (MHz)')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Residuals ($\mu$s)')
plt.show()

Even though our timing residuals have three types of extra noise, it's very hard to distinguish between them by eye just looking at residuals plots!

Next we'd like to look at time-correlated, or red noise. We'll generate some time correlated noise that runs along a power law spectrum using the FFT method.

In [ ]:
# function adapted from enterprise
def powerlaw(f, log10_A=-16, gamma=5, components=2, fref=1):
    df = f[1] - f[0]
    return (
        (10**log10_A) ** 2 / 12.0 / np.pi**2 * fref ** (gamma - 3) * f ** (-gamma) * df
    )

In [ ]:
Nf = 100
f_grid = np.arange(1/Tspan, Nf/Tspan, 1/Tspan)
rn_amp_spectrum = np.sqrt(powerlaw(f_grid, log10_A=np.log10(red_noise_A_true), gamma=red_noise_gam_true, fref=fref))

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(f_grid, rn_amp_spectrum, '.C0')
plt.xlabel('Frequency (1/yr)')
plt.ylabel(r'Amplitude spectrum ($\mu$s)')
plt.loglog()
plt.show()

We can generate a unique time series from this as the Fourier series of this spectrum

In [ ]:
# random phases
rn_phases = np.random.uniform(0, 2*np.pi, size=Nf)
# get time series
rn_t_series = 0
for a, f, phase in zip(rn_amp_spectrum, f_grid, rn_phases):
    rn_t_series += a*np.cos(2*np.pi*f*toas + phase)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(toas, rn_t_series, '.C0')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Red Noise Induced Timing Residual ($\mu$s)')
plt.show()

As an illustration we can see how the time series changes as a function of the input value of the spectral index gamma:

In [ ]:
gammas = np.flip(np.array([0,1,2,4,6]))
fig, ax = plt.subplots(len(gammas), 1, figsize=(8,2*len(gammas)),sharex=True)
for i, gam in enumerate(gammas):
    temp_spectrum = np.sqrt(powerlaw(f_grid, log10_A=np.log10(red_noise_A_true), gamma=gam, fref=fref))
    phases = np.random.uniform(0, 2*np.pi, size=Nf)
    rn_t_series_temp = 0
    for a, f, phase in zip(temp_spectrum, f_grid, phases):
        rn_t_series_temp += a*np.cos(2*np.pi*f*toas + phase)
    ax[i].plot(toas, rn_t_series_temp, '.C0', label=fr'$\gamma = {gam}$')
    ax[i].set_ylabel(fr'Red Noise ($\mu$s)')
    ax[i].legend()
ax[-1].set_xlabel('TOAs (yr)')
plt.subplots_adjust(hspace=0)
plt.show()

Finally, let's generate some chromatic noise. The time series will be generated the exact same way as the achromatic red noise, but we'll introduce a radio frequency scaling $\Delta t \propto \nu^{-2}$

In [ ]:
# get spectrum at 1400 MHz
dm_amp_spectrum = np.sqrt(powerlaw(f_grid, log10_A=np.log10(dm_noise_A_true), gamma=dm_noise_gam_true, fref=fref))
# random phases
dm_phases = np.random.uniform(0, 2*np.pi, size=Nf)
# get time series
dm_t_series = 0
for a, f, phase in zip(dm_amp_spectrum, f_grid, dm_phases):
    dm_t_series += a*np.cos(2*np.pi*f*toas + phase)
# apply scaling
dm_t_series *= (1400/freqs)**2

In [ ]:
plt.figure(figsize=(8,4))
# plot the data as a scatterplot with colors on top to show radio frequencies
sc = plt.scatter(toas, dm_t_series, c=freqs, cmap='Spectral', alpha=1, s=10,
                 vmin=nu_min, vmax=nu_max, zorder=2)
cbar = plt.colorbar(mappable=sc)
cbar.ax.set_ylabel('Radio frequency (MHz)')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Residuals ($\mu$s)')
plt.show()

Having data at multiple frequencies helps us distinguish DM noise from achromatic red noise! Otherwise we could never disentangle the effects of the two from the timing residuals.

Finally, let's put all of our noises together.

In [ ]:
resids_noisy = resids_with_white_noise + rn_t_series + dm_t_series

# plot the data with errorbars
plt.figure(figsize=(8,4))
plt.errorbar(toas, resids_noisy, yerr=toaerr, fmt='.k', alpha=0.3, ms=10, zorder=1)
# plot the data as a scatterplot with colors on top to show radio frequencies
sc = plt.scatter(toas, resids_noisy, c=freqs, cmap='Spectral', alpha=1, s=10,
                 vmin=nu_min, vmax=nu_max, zorder=2)
cbar = plt.colorbar(mappable=sc)
cbar.ax.set_ylabel('Radio frequency (MHz)')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Residuals ($\mu$s)')
plt.show()

Next what we have to do is solve the data analysis problem of disentangling these effects.

# Bayesian analysis

We are going to use the `enterprise` pulsar timing analysis package to run a noise analysis on this data. Note that we have not done any actual pulsar timing, but we can still use this toy noisy dataset to show off some of what enterprise can do. Particularly, we've glossed over the fact that we will have to vary the timing model as we fit for the noise parameters. For simplicity, we will continue ignoring this for now, but we'll show how this is accounted for when we move on to real data.

## enterprise setup

First we'll define a simplified version of `enterprise.pulsar.Pulsar` which is used to take in the data.

In [ ]:
class FakePulsar(BasePulsar):
    '''
    Inputs:
    name of the pulsar
    toas in units of seconds
    residuals in units of seconds
    toaerrs in units of seconds
    freqs in units of MHz
    '''
    def __init__(self, name, toas, residuals, toaerrs, freqs, sort=True):
        self._sort = sort

        # these are TDB but not barycentered
        self.name = name
        self._toas = toas
        self._residuals = residuals
        self._toaerrs = toaerrs
        self._ssbfreqs = freqs
        
        ntoas = len(self._toas)
        
        # add fake telescope names
        self._telescope = np.array(['simulated']*ntoas)

        # add fake flags/metadata for noise modeling
        self._flags = {}
        self._flags['pta'] = np.array(['Fake_PTA']*ntoas)
        self._flags['f'] = np.array(['toy_data']*ntoas)
        self._flags['group'] = np.array(['toy_data']*ntoas)

        # convert flags to arrays
        # TODO probably better way to do this
        #      -- PINT always stores flags as strings
        for key, val in self._flags.items():
            if isinstance(val[0], u.quantity.Quantity):
                self._flags[key] = np.array([v.value for v in val])
            else:
                self._flags[key] = np.array(val)

        self.sort_data()

In [ ]:
# create the pulsar and convert TOAs, residuals, and errors into seconds
epsr = FakePulsar('PSR', toas*365.25*86400, resids_noisy*1e-6, toaerr*1e-6, freqs)

This `FakePulsar` class is missing all of the astrophysical pieces normally contained therein. You can run `dir(epsr)` to see there are many attributes which we did not provide any means for this class to get the data for. However, the pulsar still contains the minimal data we need for noise modeling.

In [ ]:
# plotting the data again, just using the information in our pulsar class
plt.figure(figsize=(8,4))
plt.errorbar(epsr.toas, epsr.residuals, yerr=epsr.toaerrs, fmt='.k', alpha=0.3, ms=10, zorder=1)
# plot the data as a scatterplot with colors on top to show radio frequencies
sc = plt.scatter(epsr.toas, epsr.residuals, c=epsr.freqs, cmap='Spectral', alpha=1, s=10,
                 vmin=nu_min, vmax=nu_max, zorder=2)
cbar = plt.colorbar(mappable=sc)
cbar.ax.set_ylabel('Radio frequency (MHz)')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Residuals (s)')
plt.show()

enterprise will natively interact with this pulsar class to setup the likelihood for our analysis. Next we can use enterprise to define the model components in a similar way to what we can use in the analysis of IPTA data (white noise, red noise, and DM variations). Specifically, we'll fit for our parameters using the following uniform and log-uniform priors:
- EFAC: $\mathcal{U}(0.01, 10)$
- EQUAD: $\log_{10}\mathcal{U}(10^{-9}, 10^{-4})$
- ECORR: $\log_{10}\mathcal{U}(10^{-9}, 10^{-4})$
- $A_{RN}$: $\log_{10}\mathcal{U}(10^{-20}, 10^{-11})$
- $\gamma_{RN}$: $\mathcal{U}(0, 7)$
- $A_{DM}$: $\log_{10}\mathcal{U}(10^{-20}, 10^{-11})$
- $\gamma_{DM}$: $\mathcal{U}(0, 7)$

Note that log-uniform priors are uniform in log-space. These are useful as uniformative priors if you are not sure if the modeled process is actually present in the data or not.

In [ ]:
# parameters
efac = parameter.Uniform(0.01,10)
log10_equad = parameter.Uniform(-9,-4)
log10_ecorr = parameter.Uniform(-9,-4)
red_noise_log10_A = parameter.Uniform(-20,-11)
red_noise_gamma = parameter.Uniform(0,7)
dm_noise_log10_A = parameter.Uniform(-20,-11)
dm_noise_gamma = parameter.Uniform(0,7)

In [ ]:
# get our white noise signals
ef = white_signals.MeasurementNoise(efac=efac)
eq = white_signals.TNEquadNoise(log10_tnequad=log10_equad)
ec = white_signals.EcorrKernelNoise(log10_ecorr)

# define our red noise process
pl = gp_priors.powerlaw(log10_A=red_noise_log10_A, gamma=red_noise_gamma)
rn = gp_signals.FourierBasisGP(pl, components=30, name='red_noise')

# define our dm noise process
pl = gp_priors.powerlaw(log10_A=dm_noise_log10_A, gamma=dm_noise_gamma)
dm_basis = utils.createfourierdesignmatrix_dm(nmodes=100)
dm = gp_signals.BasisGP(pl, dm_basis, name='dm_gp')

# total signal collection
s = ef + eq + ec + rn + dm

In [ ]:
# Now act on pulsar object and create the PTA
pta = signal_base.PTA(s(epsr))

This PTA object has everything we need to complete the analysis. We can call `pta.summary` to get a printout of the model.

In [ ]:
print(pta.summary())

Importantly, it can compute the likelihood of our data given a set of input noise parameters.

In [ ]:
def call_log_likelihood(pta):
    # generate a random parameter vector from the prior
    params = {p.name:p.sample() for p in pta.params}
    return pta.get_lnlikelihood(params)

In [ ]:
call_log_likelihood(pta)

Check how long the likelihood call takes on your machine (should be a few ms)

In [ ]:
%timeit call_log_likelihood(pta)

## PTMCMCSampler setup

We have our likelihood, now we can run an MCMC analysis using PTMCMCSampler. enterprise_extensions contains some nice tools to set this up for us.

In [ ]:
outdir = './toy_modelnoise_chains'
if not os.path.isdir(outdir):
    os.mkdir(outdir)

In [ ]:
gr = get_parameter_groups(pta)

In [ ]:
sampler = setup_sampler(pta, outdir = outdir, groups = gr)

We can run the sampler in notebook. Thankfully, a 7 parameter model will not take long to sample. Using 100000 iterations, it should take about 5 minutes. Our output parameter samples will be saved to an output file

In [ ]:
# now we can sample - chains will be saved to outdir
Niter = 100000
x0 = np.hstack([p.sample() for p in pta.params])
x0

In [ ]:
sampler.sample(x0, Niter, burn=3_000, thin=10, SCAMweight=200, AMweight=100, DEweight=200)

# Post processing

Now we can evalute the results of our model to see if it make sense! We will use the `la_forge` package to do our post processing which contains lots of nice tools

In [ ]:
# automatically burn the first 25% of the chain when we load
core = co.Core(chaindir=outdir, params=pta.param_names, burn=0.25)

First let's check the chain traces. The parameters should each look like fuzzy caterpillars if the analysis has converged. There are other most advanced types of checks we can perform on the chains to determine convergence if we need it (R hat statistic, autocorrelation lengths)

In [ ]:
dg.plot_chains(core, hist=False, ncols=4)

Next let's see the actual parameter distributions.

In [ ]:
dg.plot_chains(core, hist=True, ncols=4)

We are using simulated data, so let's compare against the injected values as well. We can visualize this really nicely in the form of a corner plot.

In [ ]:
# here we (attempt to) convert to enterprise units for the amp parameters
truths = {
    'PSR_dm_gp_gamma': dm_noise_gam_true,
    'PSR_dm_gp_log10_A': np.log10((dm_noise_A_true*1e-6)**2/12/np.pi**2),
    'PSR_efac': efac_true,
    'PSR_log10_ecorr': np.log10(ecorr_true*1e-6),
    'PSR_basis_ecorr_log10_ecorr': np.log10(ecorr_true*1e-6),
    'PSR_log10_tnequad': np.log10(equad_true*1e-6),
    'PSR_red_noise_gamma': red_noise_gam_true,
    'PSR_red_noise_log10_A': np.log10((red_noise_A_true*1e-6)**2/12/np.pi**2)
}

In [ ]:
truths

In [ ]:
params = ['PSR_efac', 'PSR_log10_ecorr', 'PSR_log10_tnequad',
          'PSR_dm_gp_gamma', 'PSR_dm_gp_log10_A', 'PSR_red_noise_gamma', 'PSR_red_noise_log10_A']
pidxs = [core.params.index(param) for param in params]
plabels = [r'$\mathcal{F}$', r'$\log_{10}\mathcal{Q}$', r'$\log_{10}\mathcal{J}$',
           r'$\gamma_{\mathrm{DM}}$', r'$\log_{10}A_{\mathrm{DM}}$',
           r'$\gamma_{\mathrm{RN}}$', r'$\log_{10}A_{\mathrm{RN}}$']
figure = corner.corner(core.chain[core.burn:,pidxs], labels=plabels)

truths_array = np.array([truths[param] for param in params])
corner.overplot_lines(figure, truths_array, color="C0")
corner.overplot_points(figure, truths_array[None], marker="s", color="C0")
plt.show()

We did not get a perfect match to all the parameters, but it's pretty good for a crude analysis!

**Exercise for the reader:** The DM noise amplitude appears to be higher than the injected value. This may actually be a mismatch between the units in enterprise and the units I use in the notebook. Can you find what the issue is?

Finally, as we started by doing analysis of our timing residuals, it would be satisfying to check if the signals we are modeling actually match up with the signals we put into the data. We can do this in a straightforward way using `enterprise.utils.ConditionalGP`

In [ ]:
# We remake the PTA here using EcorrBasisModel which will let us compute the GPs
# You should verify that the likelihood is the same using both PTAs
ec_basis = gp_signals.EcorrBasisModel(log10_ecorr)
s_basis_ecorr = ef + eq + ec_basis + rn + dm
pta_basis_ecorr = signal_base.PTA(s_basis_ecorr(epsr))

In [ ]:
# Note using basis ecorr changes the ecorr parameter name in the pta object
pta_basis_ecorr.params

In [ ]:
gp = utils.ConditionalGP(pta_basis_ecorr)

Below we just input the true values 

**Exercise for the reader:** What happens if you add the max likleihood parameters for your noise analysis instead? The mean parameters? If you'd like to be fancy, you can setup a for loop to generate different realizations to sample from the posterior.

In [ ]:
GPs = gp.get_mean_processes(truths)

In [ ]:
GPs.keys()

In [ ]:
# plotting the data again, just using the information in our pulsar class
plt.figure(figsize=(8,4))
plt.errorbar(epsr.toas, epsr.residuals, yerr=epsr.toaerrs, fmt='.k', alpha=0.2, ms=10, zorder=1)
plt.plot(epsr.toas, GPs['PSR_red_noise'], '-C3')
plt.plot(epsr.toas, GPs['PSR_dm_gp'], '-C1')
plt.plot(epsr.toas, GPs['PSR_basis_ecorr'], '-C0')
plt.xlabel('TOAs (yr)')
plt.ylabel(r'Residuals (s)')
plt.show()

In [ ]:
all_gps = GPs['PSR_red_noise'] + GPs['PSR_dm_gp'] + GPs['PSR_basis_ecorr']

In [ ]:
toaerrs_adjusted = np.sqrt(truths['PSR_efac']**2*epsr.toaerrs**2 + (10**truths['PSR_log10_tnequad'])**2)

In [ ]:
# plotting the data again, just using the information in our pulsar class
fig, ax = plt.subplots(3,1,figsize=(8,7),sharex=True)
ax[0].errorbar(epsr.toas, epsr.residuals, yerr=epsr.toaerrs, fmt='.k', alpha=0.2, ms=10, zorder=1)
ax[0].plot(epsr.toas, all_gps, '-C2')
ax[0].set_ylabel(r'Residuals (s)')
ax[1].errorbar(epsr.toas, epsr.residuals - all_gps, yerr=epsr.toaerrs, fmt='.k', alpha=0.2, ms=10, zorder=1)
ax[1].set_ylabel(r'Whitened Residuals (s)')
ax[2].scatter(epsr.toas, (epsr.residuals - all_gps)/toaerrs_adjusted, c='k', alpha=0.2, s=10, zorder=1)
ax[2].set_ylabel(r'Normalized Residuals')
ax[2].set_xlabel('TOAs (yr)')
plt.subplots_adjust(hspace=0)
plt.show()